# Cálculo de Diferencia de Diferencias (DifDif)

Este notebook calcula nuevas variables que miden la **diferencia de diferencias** entre Generales y Ballotage.

## Concepto:

Para cada ítem, calculamos:

1. **Diferencia en Generales**: `Dif_Gen = Izq - Der` (en Generales)
2. **Diferencia en Ballotage**: `Dif_Bal = Izq - Der` (en Ballotage)
3. **Diferencia de Diferencias**: `DifDif = Dif_Bal - Dif_Gen`

Esto nos dice **cómo cambió la asimetría izquierda-derecha** entre ambas elecciones.

## Variables calculadas:

- `DifDif_CO_Item_X`: Para Cambio de Opinión
- `DifDif_CT_Item_X`: Para Cambio de Tiempo

Donde X es el número de ítem (3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30)

In [ ]:
import pandas as pd
import numpy as np
import os

print("Librerías cargadas exitosamente")

## 1. Cargar Datos

In [ ]:
# Rutas a los archivos Excel
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames desde Excel
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

print(f"✓ Datos cargados desde Excel:")
print(f"  - Generales: {len(df_Generales)} registros, {len(df_Generales.columns)} columnas")
print(f"  - Ballotage: {len(df_Ballotage)} registros, {len(df_Ballotage.columns)} columnas")

## 2. Definir Ítems a Procesar

In [ ]:
# Ítems progresistas y conservadores
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]

# Todos los ítems
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

print(f"Total de ítems a procesar: {len(Todos_Items)}")
print(f"Ítems: {Todos_Items}")

## 3. Función para Calcular Diferencia de Diferencias

In [ ]:
def Calcular_DifDif_Para_Tipo(df_gen, df_bal, items, tipo):
    """
    Calcula las variables de diferencia de diferencias para un tipo específico (CO o CT).
    
    Parámetros:
    - df_gen: DataFrame de Generales
    - df_bal: DataFrame de Ballotage
    - items: Lista de números de ítems
    - tipo: 'CO' o 'CT'
    
    Retorna:
    - DataFrame con las columnas DifDif calculadas
    """
    
    # DataFrame resultado
    df_resultado = pd.DataFrame()
    
    # Para cada ítem
    for item in items:
        # Nombres de las variables
        var_izq = f'{tipo}_Item_{item}_Izq'
        var_der = f'{tipo}_Item_{item}_Der'
        
        # Verificar que las columnas existen en ambos DataFrames
        if var_izq in df_gen.columns and var_der in df_gen.columns and \
           var_izq in df_bal.columns and var_der in df_bal.columns:
            
            # Paso 1: Diferencia en Generales (Izq - Der)
            Dif_Gen = df_gen[var_izq] - df_gen[var_der]
            
            # Paso 2: Diferencia en Ballotage (Izq - Der)
            Dif_Bal = df_bal[var_izq] - df_bal[var_der]
            
            # Paso 3: Diferencia de Diferencias
            DifDif = Dif_Bal - Dif_Gen
            
            # Nombre de la variable resultado
            nombre_dif_gen = f'Dif_Gen_{tipo}_Item_{item}'
            nombre_dif_bal = f'Dif_Bal_{tipo}_Item_{item}'
            nombre_difdif = f'DifDif_{tipo}_Item_{item}'
            
            # Agregar al DataFrame resultado
            df_resultado[nombre_dif_gen] = Dif_Gen
            df_resultado[nombre_dif_bal] = Dif_Bal
            df_resultado[nombre_difdif] = DifDif
            
        else:
            print(f"⚠️  Variables {var_izq} o {var_der} no encontradas en ambos DataFrames")
    
    return df_resultado

## 4. Calcular DifDif para CO (Cambio de Opinión)

In [ ]:
print("Calculando DifDif para CO (Cambio de Opinión)...\n")

df_DifDif_CO = Calcular_DifDif_Para_Tipo(
    df_gen=df_Generales,
    df_bal=df_Ballotage,
    items=Todos_Items,
    tipo='CO'
)

print(f"\n✓ Calculadas {len(df_DifDif_CO.columns)} variables para CO")
print(f"  Total de filas: {len(df_DifDif_CO)}")
print(f"\nPrimeras columnas: {df_DifDif_CO.columns[:6].tolist()}")

## 5. Calcular DifDif para CT (Cambio de Tiempo)

In [ ]:
print("Calculando DifDif para CT (Cambio de Tiempo)...\n")

df_DifDif_CT = Calcular_DifDif_Para_Tipo(
    df_gen=df_Generales,
    df_bal=df_Ballotage,
    items=Todos_Items,
    tipo='CT'
)

print(f"\n✓ Calculadas {len(df_DifDif_CT.columns)} variables para CT")
print(f"  Total de filas: {len(df_DifDif_CT)}")
print(f"\nPrimeras columnas: {df_DifDif_CT.columns[:6].tolist()}")

## 6. Crear DataFrame Elecciones Completo

In [ ]:
# Combinar ambos DataFrames
df_Elecciones = pd.concat([df_DifDif_CO, df_DifDif_CT], axis=1)

# Agregar información de identificación desde df_Generales
# Asumiendo que ambos DataFrames tienen el mismo número de filas en el mismo orden
columnas_identificacion = ['Categoria_PASO_2023']

# Agregar más columnas si existen
if 'ID' in df_Generales.columns:
    columnas_identificacion.insert(0, 'ID')

for col in columnas_identificacion:
    if col in df_Generales.columns:
        df_Elecciones.insert(0, col, df_Generales[col].values)

print(f"✓ DataFrame df_Elecciones creado exitosamente")
print(f"  Dimensiones: {df_Elecciones.shape}")
print(f"  Total de variables DifDif: {len([col for col in df_Elecciones.columns if col.startswith('DifDif')])}")
print(f"\nColumnas de identificación: {[col for col in df_Elecciones.columns if not col.startswith('Dif')]}")

## 7. Exploración de los Datos

In [ ]:
# Mostrar las primeras filas
print("Primeras 5 filas de df_Elecciones:\n")
print(df_Elecciones.head())

In [ ]:
# Estadísticas descriptivas de las variables DifDif_CO
columnas_difdif_co = [col for col in df_Elecciones.columns if col.startswith('DifDif_CO')]

print(f"\nEstadísticas descriptivas de DifDif_CO (primeras 5 variables):\n")
print(df_Elecciones[columnas_difdif_co[:5]].describe())

In [ ]:
# Estadísticas descriptivas de las variables DifDif_CT
columnas_difdif_ct = [col for col in df_Elecciones.columns if col.startswith('DifDif_CT')]

print(f"\nEstadísticas descriptivas de DifDif_CT (primeras 5 variables):\n")
print(df_Elecciones[columnas_difdif_ct[:5]].describe())

## 8. Verificar Valores Faltantes

In [ ]:
# Contar valores faltantes
print("Valores faltantes por variable DifDif_CO:\n")
valores_faltantes_co = df_Elecciones[columnas_difdif_co].isnull().sum()
print(valores_faltantes_co[valores_faltantes_co > 0])
if valores_faltantes_co.sum() == 0:
    print("  ✓ No hay valores faltantes en DifDif_CO")

In [ ]:
print("\nValores faltantes por variable DifDif_CT:\n")
valores_faltantes_ct = df_Elecciones[columnas_difdif_ct].isnull().sum()
print(valores_faltantes_ct[valores_faltantes_ct > 0])
if valores_faltantes_ct.sum() == 0:
    print("  ✓ No hay valores faltantes en DifDif_CT")

## 9. Distribución por Categoría

In [ ]:
# Contar registros por categoría
if 'Categoria_PASO_2023' in df_Elecciones.columns:
    print("\nDistribución de registros por Categoría PASO 2023:\n")
    print(df_Elecciones['Categoria_PASO_2023'].value_counts().sort_index())
    print(f"\nTotal: {df_Elecciones['Categoria_PASO_2023'].value_counts().sum()} registros")

## 10. Guardar DataFrame Elecciones

In [ ]:
# Crear carpeta si no existe
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar en Excel
Ruta_Salida = os.path.join(Carpeta_Salida, 'df_Elecciones.xlsx')
df_Elecciones.to_excel(Ruta_Salida, index=False)

print(f"\n✅ DataFrame guardado exitosamente en: {Ruta_Salida}")
print(f"   {len(df_Elecciones)} registros × {len(df_Elecciones.columns)} columnas")

## 11. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN DEL CÁLCULO DE DIFERENCIA DE DIFERENCIAS")
print("="*70)
print(f"\n📊 Variables calculadas:")
print(f"   - Variables DifDif_CO: {len(columnas_difdif_co)}")
print(f"   - Variables DifDif_CT: {len(columnas_difdif_ct)}")
print(f"   - Total variables DifDif: {len(columnas_difdif_co) + len(columnas_difdif_ct)}")

print(f"\n📁 DataFrame df_Elecciones:")
print(f"   - Dimensiones: {df_Elecciones.shape[0]} filas × {df_Elecciones.shape[1]} columnas")
print(f"   - Archivo guardado: df_Elecciones.xlsx")

print(f"\n🎯 Interpretación de DifDif:")
print(f"   - DifDif > 0: La diferencia Izq-Der aumentó en Ballotage")
print(f"   - DifDif < 0: La diferencia Izq-Der disminuyó en Ballotage")
print(f"   - DifDif = 0: No hubo cambio en la asimetría Izq-Der")

print(f"\n✅ Proceso completado exitosamente")
print("="*70)